# RUN SfM Pipeline with Deep-Image-Matching


## Initialization

Let's first import DIM and load a logger to see what's going on


In [10]:
from pprint import pprint

import yaml

import deep_image_matching as dim

logger = dim.setup_logger("dim")

### Define the configuration

Get the list of possible pipelines and matching strategy and chose one of them.


In [11]:
print("Available pipelines:")
pprint(dim.Config.get_pipelines())
print("Available matching strategy:")
pprint(dim.Config.get_matching_strategies())

Available pipelines:
['superpoint+lightglue',
 'superpoint+lightglue_fast',
 'superpoint+superglue',
 'superpoint+kornia_matcher',
 'disk+lightglue',
 'aliked+lightglue',
 'orb+kornia_matcher',
 'sift+kornia_matcher',
 'loftr',
 'se2loftr',
 'roma',
 'keynetaffnethardnet+kornia_matcher',
 'dedode+kornia_matcher']
Available matching strategy:
['bruteforce',
 'sequential',
 'retrieval',
 'custom_pairs',
 'matching_lowres',
 'covisibility']


Now you have to build a dictionary with the input processing parameters (they are the same as the input parameters for the CLI) and pass it to the Config class to initialize the configuration object. Refer to the [documentation](https://3dom-fbk.github.io/deep-image-matching/) for more information about the parameters.

Note that the `dir` defines the project directory, where the images are stored and the results will be saved.
Deep-Image-Matching will search for the images inside an 'image' subdirectory and will save the results in a 'results\_{processing_params}' subdirectory, where {processing_params} are some information on the processing parameters used.

By default DIM will not run if the output directory already exists, to avoid overwriting previous results. If you want to overwrite the results, you can set the `force` parameter to True. We have not implemented the possibility to recover the previous results yet (e.g., by using existing extracted features), but we may add it in the future.

The `config_file` parameter is the path to the configuration file (optional). In this file you can specify all the parameters that you need for controlling the feature extraction and matching. Refer to the [documentation](https://3dom-fbk.github.io/deep-image-matching/advanced_configuration/) for more information about how to write this file. Note that this file il optional, if you don't pass it, the default parameters will be used.

If you use set `verbose` to True, DIM will log all the processing steps and it will save some figures with the extracted features and the matches in a `debug` folder inside the results directory. Note that this will slow down the processing and will create a lot of files if the dataset is big. It is reccomended to use it only for testing or debugging purposes.


In [12]:
params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "config_file": ".../assets/example_cyprus/config_superpoint+lightglue.yaml",
    "strategy": "matching_lowres",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": False,
    "force": True,
    "camera_options": "../config/cameras.yaml",
    "openmvg": None,
    "verbose": False,
}
config = dim.Config(params)

# Save the configuration to a json file for later use
config.save()

2025-07-04 12:25:03 | [WARNING ] ../assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high already exists, but the '--force' option is used. Deleting the folder.


FileNotFoundError: Configuration file /home/francesco/dev/deep-image-matching/notebooks/.../assets/example_cyprus/config_superpoint+lightglue.yaml not found.

You can check the configuration object.


In [ ]:
print("Config general:")
pprint(config.general)
print("Config extractor:")
pprint(config.extractor)
print("Config matcher:")
pprint(config.matcher)

## Run the extraction and matching

First, you have to create an instance of the ImageMatching class and pass the configuration object to it.

Then you can run the extraction and matching by calling the `run` method.
This method will automatically run all the steps needed to extract the features and match the images. It will return the path to the h5 files containing the features and the matches.
The `features.h5` file contains the features extracted from each images, while the `matches.h5` file contains the indices of the features matched.


In [ ]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

#### The `features.h5` file

#### The `matches.h5` file


In [ ]:
# # print features_h5 content
# with h5py.File(features_h5, "r") as f:
#     print(f.keys())
#     print(f[images[0].name].keys())
#     print(f[images[0].name]["keypoints"][:])
#     print(f[images[1].name].keys())
#     print(f[images[1].name]["keypoints"][:])

# # Print matches.h5 content
# with h5py.File(matches_h5, "r") as f:
#     print(f.keys())
#     g0 = f[images[0].name]
#     print(g0.keys())
#     g1 = g0[images[1].name]
#     print(g1.__array__())

## Export in colmap format

DIM assigns camera models to images based on the options defined in `cameras.yaml` file. You can read this file with the `yaml` library which returns a dictionary.

Then you can use the `export_to_colmap` function that will read the features and the matches from the h5 files and will save them in a COLMAP sqliite database. If you pass the `cameras` dictionary, it will also save the camera models in the database.


In [ ]:
# Read camera options
with open(config.general["camera_options"]) as file:
    camera_options = yaml.safe_load(file)

database_path = config.general["output_dir"] / "database.db"
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)

Alternatively you can assign camera models with a dictionary.

For images not assigned to specific `cam<x>` camera groups, the options specified under `general` are applied. The `camera_model` can be selected from `["simple-pinhole", "pinhole", "simple-radial", "opencv"]`. It's worth noting that it's easily possible to extend this to include all the classical COLMAP camera models. Cameras can either be shared among all images (`single_camera == True`), or each camera can have a different camera model (`single_camera == False`).

A subset of images can share intrinsics using `cam<x>` key, by specifying the `camera_model` along with the names of the images separated by commas. There's no limit to the number of `cam<x>` entries you can use.

**Note**: Use the SIMPLE-PINHOLE camera model if you want to export the solution to Metashape later, as there are some bugs in COLMAP (or pycolamp) when exportingthe solution in the Bundler format.
e.g., using FULL-OPENCV camera model, the principal point is not exported correctly and the tie points are wrong in Metashape.

```python
camera_options = {
    "general": {
        "camera_model": "pinhole",  # ["simple-pinhole", "pinhole", "simple-radial", "opencv"]
        "single_camera": True,
    },
    "cam0": {
        "camera_model": "pinhole",
        "images": "DSC_6468.JPG,DSC_6468.JPG",
    },
    "cam1": {
        "camera_model": "pinhole",
        "images": "",
    },
}
```


## Run reconstruction

You can run the reconstruction with pycolmap, OpenMVG or MICMAC. The suggested method is pycolmap, as it is the most integrated with DIM.


If the pycolmap module is imported, you can define all the parameters for the COLAMP reconstruction.
You can check all the available parameters with:

```python
import pycolmap

print(pycolmap.IncrementalPipelineOptions().summary())
```

To define the reconstruction options, you can create an instance of the `IncrementalPipelineOptions` class and set the parameters you want to change. The default values are already set in the class, so you only need to set the parameters you want to change.

```python
opts = pycolmap.IncrementalPipelineOptions()
opts.triangulation.ignore_two_view_tracks = False
opts.mapper.filter_min_tri_angle = 0.5
opts.mapper.filter_max_reproj_error = 5.0
```

Alternatively, you can create a dictionary with the options you want to change. The keys of the dictionary should be the names of the parameters and the values should be the values you want to set. You can then pass this dictionary to the `incremental_reconstruction` function.

```python
reconst_opts = {
    "triangulation": {
        "ignore_two_view_tracks": False,
    },
    "mapper": {
        "filter_min_tri_angle": 0.5,
        "filter_max_reproj_error": 5.0,
    },
}
```

The function `dim.reconstruction.incremental_reconstruction` exposes some of the parameters of the `pycolmap.IncrementalPipelineOptions` class, so you can pass them directly to the function. For example, you can set the `refine_intrinsics` parameter to `True` to refine the intrinsics during the reconstruction.

```python
reconstruction = dim.reconstruction.incremental_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=True,
    ignore_two_view_tracks=True,
    reconstruction_options=opts,
)
```

If you want to use the default options, you can just pass `None` or an empty dictionary to the `incremental_reconstruction` function.

```python
reconstruction = dim.reconstruction.incremental_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    image_dir=config.general["image_dir"],
    sfm_dir=config.general["output_dir"],
    reconstruction_options=None,
)
```

The `incremental_reconstruction` function will return a `pycolmap.Reconstruction` object that contains the reconstruction results. You can use this object to access the cameras, images, and points of the reconstruction.

```python
print(reconstruction.summary())
```


In [ ]:
import pycolmap

print(pycolmap.IncrementalPipelineOptions().summary())

IncrementalPipelineOptions:
    min_num_matches = 15
    ignore_watermarks = False
    multiple_models = True
    max_num_models = 50
    max_model_overlap = 20
    min_model_size = 10
    init_image_id1 = -1
    init_image_id2 = -1
    init_num_trials = 200
    extract_colors = True
    num_threads = -1
    min_focal_length_ratio = 0.1
    max_focal_length_ratio = 10.0
    max_extra_param = 1.0
    ba_refine_focal_length = True
    ba_refine_principal_point = False
    ba_refine_extra_params = True
    ba_refine_sensor_from_rig = True
    ba_min_num_residuals_for_cpu_multi_threading = 50000
    ba_local_num_images = 6
    ba_local_function_tolerance = 0.0
    ba_local_max_num_iterations = 25
    ba_global_frames_ratio = 1.1
    ba_global_points_ratio = 1.1
    ba_global_frames_freq = 500
    ba_global_points_freq = 250000
    ba_global_function_tolerance = 0.0
    ba_global_max_num_iterations = 50
    ba_local_max_refinements = 2
    ba_local_max_refinement_change = 0.001
    ba_globa

In [ ]:
opts = pycolmap.IncrementalPipelineOptions()
opts.triangulation.ignore_two_view_tracks = False
opts.triangulation.min_angle = 0.5
opts.mapper.filter_min_tri_angle = 0.5
opts.mapper.filter_max_reproj_error = 5.0

In [ ]:
# Run reconstruction
refine_intrinsics = False
model = dim.reconstruction.incremental_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=refine_intrinsics,
    reconstruction_options=opts,
)

NameError: name 'dim' is not defined

In [ ]:
# Print COLMAP camera values
print(list(model.cameras.values()))